# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

An error occured.
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /var/folders/nx/j2m2blq56dlbrdcf0g7dxt_c0000gq/T/jupyterlab-debug-kx3rur_o.log


In [2]:
!pip install python-gmaps

In [3]:
!jupyter lab build

[LabBuildApp] JupyterLab 1.2.6
[LabBuildApp] Building in /opt/anaconda3/share/jupyter/lab
An error occured.
ValueError: Please install nodejs 5+ and npm before continuing installation. nodejs may be installed using conda or directly from the nodejs website.
See the log file for details:  /var/folders/nx/j2m2blq56dlbrdcf0g7dxt_c0000gq/T/jupyterlab-debug-qook665b.log


In [4]:
import gmaps
import gmaps.datasets

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather = ("../output_data/cities.csv")

weather_df = pd.read_csv(weather)

weather_df.head(10)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
import gmaps
import gmaps.datasets

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [8]:
gmaps.configure(api_key = g_key)

locations = weather_df[['Lat', 'Lng']]

weights = weather_df['Humidity']

fig = gmaps.figure()

fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig            

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
cleanedweather_df = weather_df

cleanedweather_df = cleanedweather_df.loc[(cleanedweather_df["Max Temp"] <= 85) & (cleanedweather_df["Max Temp"] > 70)]
cleanedweather_df = cleanedweather_df.loc[cleanedweather_df["Wind Speed"] < 15]
cleanedweather_df = cleanedweather_df.loc[cleanedweather_df["Cloudiness"] == 0]
                                                                                     
cleanedweather_df = cleanedweather_df.dropna()

cleanedweather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = cleanedweather_df

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row ["Lng"]

    params["location"]= f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response ['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing result..skipping")
    
    print("-----------")

Closest hotel is Hostal San Isidro Oficial.
-----------
Closest hotel is Escarpas do Lago.
-----------
Closest hotel is Novomar Pousada e Turismo Ltda.
-----------
Closest hotel is Le Monitel.
-----------
Closest hotel is Pousada Porto Praia.
-----------
Closest hotel is La Case Bambou.
-----------
Closest hotel is Dandeli Lake County.
-----------
Closest hotel is Hôtel La Giralda.
-----------
Closest hotel is MOGOK HILL HOTEL.
-----------
Closest hotel is Boboi Beach Lodge.
-----------
Closest hotel is Hotel Relax Inn.
-----------
Closest hotel is تاکسی سرویس آرامش.
-----------
Closest hotel is Marruá Hotel.
-----------
Closest hotel is Norma franco.
-----------
Closest hotel is صالون مها حيدر - Salon Maha.
-----------


In [11]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Hostal San Isidro Oficial
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Novomar Pousada e Turismo Ltda
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Le Monitel
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Pousada Porto Praia
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17,Boboi Beach Lodge


In [13]:
!jupyter nbextension enable --py gmaps
locations = hotel_df[["Lat", "Lng"]]
m =gmaps.figure()

markers = gmaps.marker_layer(locations)
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(150, 0, 0, 0.9)',
    stroke_color='rgba(0, 0, 150, 0.9)', scale=3,
    info_box_content=hotel_info
)
m.add_layer(markers)
m.add_layer(hotel_layer)

m

Traceback (most recent call last):
  File "/opt/anaconda3/bin/jupyter-nbextension", line 11, in <module>
    sys.exit(main())
  File "/opt/anaconda3/lib/python3.7/site-packages/jupyter_core/application.py", line 268, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.7/site-packages/notebook/nbextensions.py", line 983, in start
    super(NBExtensionApp, self).start()
  File "/opt/anaconda3/lib/python3.7/site-packages/jupyter_core/application.py", line 257, in start
    self.subapp.start()
  File "/opt/anaconda3/lib/python3.7/site-packages/notebook/nbextensions.py", line 891, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/opt/anaconda3/lib/python3.7/site-packages/notebook/nbextensions.py", line 867, in toggle_nbextension_python
    logger=self.log)
  File "/opt

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add marker layer ontop of heat map


# Display figure